In [10]:
#importing the libraries needed
import pandas as pd
import requests, json
from pprint import pprint
pd.set_option('display.max_columns', None)

In [6]:
#base url for the endpoints
base_url = 'https://fantasy.premierleague.com/api/'

r = requests.get(base_url+'bootstrap-static/').json() #this is the main url for all premier league players, teams, global gameweek summaries

# show the top level fields
pprint(r, indent=2, depth=1, compact=True)

{ 'element_stats': [...],
  'element_types': [...],
  'elements': [...],
  'events': [...],
  'game_settings': {...},
  'phases': [...],
  'teams': [...],
  'total_players': 4905998}


In [11]:
# get player data from 'elements' field and creating a dataframe
players = pd.json_normalize(r['elements'])
players.head()

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,id,in_dreamteam,news,news_added,now_cost,photo,points_per_game,second_name,selected_by_percent,special,squad_number,status,team,team_code,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text,expected_goals_per_90,saves_per_90,expected_assists_per_90,expected_goal_involvements_per_90,expected_goals_conceded_per_90,goals_conceded_per_90,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90
0,NaN,None,438098,0,0,0,0,0,3,3.0,None,0,Fábio,0.0,1,False,,None,55,438098.jpg,2.2,Ferreira Vieira,0.1,False,None,a,1,3,24,0,0,0,0,0.0,4.4,Fábio Vieira,290,1,3,1,2,0,0,0,0,1,0,0,67,82.8,88.8,85.0,25.5,2,1.25,0.64,1.89,1.83,336,147,268,147,255,131,322,148,NaN,,NaN,,4.0,,0.39,0.0,0.20,0.59,0.57,0.62,145,89,477,190,251,105,454,157,0.62,0.31
1,NaN,None,205651,0,0,0,0,0,4,3.2,None,0,Gabriel,0.0,2,False,,None,70,205651.jpg,3.1,Fernando de Jesus,2.2,False,None,a,1,3,85,0,0,0,0,0.0,12.1,G.Jesus,1470,4,7,11,11,0,0,0,6,0,0,10,271,364.8,403.0,756.0,152.7,17,6.27,3.26,9.53,11.67,184,20,84,10,30,16,71,16,NaN,,NaN,,NaN,,0.38,0.0,0.20,0.58,0.71,0.67,30,13,561,56,144,22,129,22,1.04,0.67
2,NaN,None,226597,0,0,0,0,0,2,4.2,None,0,Gabriel,0.0,3,False,,None,60,226597.jpg,4.1,dos Santos Magalhães,11.9,False,None,a,1,3,149,0,0,0,0,0.0,24.8,Gabriel,3042,3,1,16,26,1,0,0,4,0,0,8,676,675.8,121.9,379.0,117.9,34,4.48,0.65,5.13,26.18,59,18,239,73,92,4,110,21,NaN,,NaN,,NaN,,0.13,0.0,0.02,0.15,0.77,0.77,82,6,512,193,43,6,34,12,1.01,0.47
3,NaN,None,219847,0,0,0,0,0,4,3.5,None,0,Kai,0.0,4,False,,None,80,219847.jpg,4.9,Havertz,13.9,False,None,a,1,3,180,0,0,0,0,0.0,22.5,Havertz,2627,13,10,16,22,0,0,0,11,0,0,15,515,763.2,613.4,1027.0,240.2,30,12.36,3.88,16.23,21.03,43,8,47,2,14,7,19,5,NaN,,NaN,,3.0,,0.42,0.0,0.13,0.55,0.72,0.75,12,4,580,67,18,4,26,6,1.03,0.55
4,NaN,None,463748,0,0,0,0,0,1,2.0,None,0,Karl,0.0,5,False,,None,40,463748.jpg,0.0,Hein,0.2,False,None,a,1,3,0,0,0,0,0,0.0,0.0,Hein,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.00,0.00,0.00,0.00,446,52,437,40,411,24,449,52,NaN,,NaN,,NaN,,0.00,0.0,0.00,0.00,0.00,0.00,570,61,56,23,448,52,350,54,0.00,0.00


In [20]:
# renaming the 'id' column to 'player id'
players.rename(columns={'id':'player_id'}, inplace=True)

In [12]:
#creating a teams dataframe
teams = pd.json_normalize(r['teams'])
teams.head()

,code,draw,form,id,loss,name,played,points,position,short_name,strength,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,3,0,None,1,0,Arsenal,0,0,0,ARS,5,None,False,0,1350,1380,1370,1370,1330,1390,1
1,7,0,None,2,0,Aston Villa,0,0,0,AVL,3,None,False,0,1120,1245,1110,1140,1130,1350,2
2,91,0,None,3,0,Bournemouth,0,0,0,BOU,3,None,False,0,1100,1100,1075,1100,1130,1105,127
3,94,0,None,4,0,Brentford,0,0,0,BRE,3,None,False,0,1100,1100,1105,1095,1100,1110,130
4,36,0,None,5,0,Brighton,0,0,0,BHA,3,None,False,0,1100,1100,1100,1105,1100,1100,131


In [9]:
# get position information from 'element_types' field
positions = pd.json_normalize(r['element_types'])
positions.head()

,id,plural_name,plural_name_short,singular_name,singular_name_short,squad_select,squad_min_select,squad_max_select,squad_min_play,squad_max_play,ui_shirt_specific,sub_positions_locked,element_count
0,1,Goalkeepers,GKP,Goalkeeper,GKP,2,None,None,1,1,True,[12],64
1,2,Defenders,DEF,Defender,DEF,5,None,None,3,5,False,[],198
2,3,Midfielders,MID,Midfielder,MID,5,None,None,2,5,False,[],259
3,4,Forwards,FWD,Forward,FWD,3,None,None,1,3,False,[],70


In [21]:

# merging the three dataframe(players, teams & positions) to one dataframe containing all the information
player_whole = pd.merge(players, teams, left_on='team', right_on='id')
player_whole = player_whole.merge(positions, left_on='element_type', right_on='id')

# selecting some few fields in the dataframe
player_whole = player_whole[['player_id', 'player_code', 'web_name', 'short_name', 'singular_name_short']]
player_whole.head()

,player_id,player_code,web_name,short_name,singular_name_short
0,1,438098,Fábio Vieira,ARS,MID
1,7,85955,Jorginho,ARS,MID
2,9,444145,Martinelli,ARS,MID
3,10,200641,Nelson,ARS,MID
4,12,499175,Nwaneri,ARS,MID


In [22]:
# get season history function using the player id
def get_season_history(player_id):
    r = requests.get(base_url + 'element-summary/' + str(player_id) + '/').json()
    df = pd.json_normalize(r['history_past'])
    return df

In [23]:
import swifter

# get gameweek histories for each player
history = player_whole['player_id'].swifter.apply(get_season_history)

Dask Apply:   0%|          | 0/9 [00:00<?, ?it/s]

In [24]:
history.head()

0      season_name  element_code  start_cost  end_c...
1      season_name  element_code  start_cost  end_c...
2      season_name  element_code  start_cost  end_c...
3      season_name  element_code  start_cost  end_c...
4      season_name  element_code  start_cost  end_c...
Name: player_id, dtype: object

In [25]:
# combine results into a single dataframe
history = pd.concat(player_whole for player_whole in history)
history.head()

,season_name,element_code,start_cost,end_cost,total_points,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded
0,2022/23,438098,60,56,40,500,1,2,2,5,0,0,0,0,0,0,2,134,116.0,180.6,123.0,41.5,3,0.86,1.39,2.25,5.28
1,2023/24,438098,55,54,24,290,1,3,1,2,0,0,0,0,1,0,0,67,82.8,88.8,85.0,25.5,2,1.25,0.64,1.89,1.83
0,2018/19,85955,50,46,96,3156,2,0,16,37,0,0,0,8,0,0,4,577,691.4,628.4,122.0,144.2,0,0.00,0.00,0.00,0.00
1,2019/20,85955,50,49,92,2376,4,3,6,40,0,0,0,10,0,0,9,535,600.0,422.4,158.0,118.4,0,0.00,0.00,0.00,0.00
2,2020/21,85955,50,47,114,2006,7,2,14,17,0,0,2,2,0,0,15,485,588.8,383.5,175.0,114.9,0,0.00,0.00,0.00,0.00


In [26]:
# merging the player whole dataframe with history dataframe
history = history.merge(player_whole, left_on='element_code', right_on='player_code')
history.drop(columns=['player_id', 'player_code', 'element_code'], axis=1, inplace=True)
history.head()

,season_name,start_cost,end_cost,total_points,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,web_name,short_name,singular_name_short
0,2022/23,60,56,40,500,1,2,2,5,0,0,0,0,0,0,2,134,116.0,180.6,123.0,41.5,3,0.86,1.39,2.25,5.28,Fábio Vieira,ARS,MID
1,2023/24,55,54,24,290,1,3,1,2,0,0,0,0,1,0,0,67,82.8,88.8,85.0,25.5,2,1.25,0.64,1.89,1.83,Fábio Vieira,ARS,MID
2,2018/19,50,46,96,3156,2,0,16,37,0,0,0,8,0,0,4,577,691.4,628.4,122.0,144.2,0,0.00,0.00,0.00,0.00,Jorginho,ARS,MID
3,2019/20,50,49,92,2376,4,3,6,40,0,0,0,10,0,0,9,535,600.0,422.4,158.0,118.4,0,0.00,0.00,0.00,0.00,Jorginho,ARS,MID
4,2020/21,50,47,114,2006,7,2,14,17,0,0,2,2,0,0,15,485,588.8,383.5,175.0,114.9,0,0.00,0.00,0.00,0.00,Jorginho,ARS,MID


In [31]:
cols = ['singular_name_short', 'short_name', 'web_name']
# moving the last three columns to the first three columns
for c in cols:
    col = history.pop(c)
    history.insert(0, c, col)

history.head()

,web_name,short_name,singular_name_short,season_name,start_cost,end_cost,total_points,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded
0,Fábio Vieira,ARS,MID,2022/23,60,56,40,500,1,2,2,5,0,0,0,0,0,0,2,134,116.0,180.6,123.0,41.5,3,0.86,1.39,2.25,5.28
1,Fábio Vieira,ARS,MID,2023/24,55,54,24,290,1,3,1,2,0,0,0,0,1,0,0,67,82.8,88.8,85.0,25.5,2,1.25,0.64,1.89,1.83
2,Jorginho,ARS,MID,2018/19,50,46,96,3156,2,0,16,37,0,0,0,8,0,0,4,577,691.4,628.4,122.0,144.2,0,0.00,0.00,0.00,0.00
3,Jorginho,ARS,MID,2019/20,50,49,92,2376,4,3,6,40,0,0,0,10,0,0,9,535,600.0,422.4,158.0,118.4,0,0.00,0.00,0.00,0.00
4,Jorginho,ARS,MID,2020/21,50,47,114,2006,7,2,14,17,0,0,2,2,0,0,15,485,588.8,383.5,175.0,114.9,0,0.00,0.00,0.00,0.00


In [32]:
#saving the dataframe
history.to_csv(r"C:\Users\HP 820 G3\Documents\solar_data\fpl_data2.csv")